## Universidad de Costa Rica
#### Facultad de Ingeniería
#### Escuela de Ciencias de la Computación e Informática


### Computabilidad y Complejidad

# Tarea Programada #2

#### Ariel Arévalo Alvarado
##### ariel.arevalo@ucr.ac.cr
#### Jorge Díaz Sagot
##### jorge.diazsagot@ucr.ac.cr

##### Ciudad Universitaria Rodrigo Facio, Costa Rica
##### II-2024

## Problema

Una organización necesita diseñar una red de comunicación robusta contra fallos como respaldo para recuperación de desastres. La construcción de esta red se modela de la siguiente forma:

Cada potencial enlace entre los nodos iniciales tiene: 
* Un costo mayor a cero
* Una calificación de fiabilidad del cero al uno

A partir de estas características, para cualquier número de nodos iniciales con potenciales enlaces, se debe construir una red donde:
* Debe existir algún camino entre cualesquiera dos nodos de la red
* Se minimice el costo total de todos los enlaces en la red
* La suma de la fiabilidad de todos los enlaces a cada nodo sea mayor o igual a uno

## Soluciones

In [75]:
import itertools
import random
import math

from src.experiment.graph import CostReliabilityGraph
from src.experiment.network import CostReliabilityNetwork

#### Fuerza Bruta

In [76]:
def brute_force(topology: CostReliabilityGraph) -> CostReliabilityNetwork:
    """
    Brute force algorithm to find the optimal network.
    """
    nodes: list = topology.nodes
    all_edges: list = topology.edges # ~O(n^2)

    best_cost = float('inf')
    best_network = None

    for r in range(len(nodes) - 1, len(all_edges) + 1):
        for edge_subset in itertools.combinations(all_edges, r): # ~O(2^n^2)
            candidate = CostReliabilityNetwork()
            candidate.add_nodes(nodes)
            candidate.add_edges(edge_subset)
            if candidate.is_feasible:
                if candidate.total_cost < best_cost:
                    best_cost = candidate.total_cost
                    best_network = candidate
    return best_network

#### Heurística

In [77]:
def heuristic(topology: CostReliabilityGraph) -> CostReliabilityNetwork:
    """
    Heuristic algorithm that builds a connected and reliable network
    by adding edges in order of ascending modified weight (cost/reliability),
    ensuring that each edge added helps in connectivity or node reliability.
    """
    nodes: list = topology.nodes
    all_edges: list = topology.edges

    processed_edges = []
    for (u, v, data) in all_edges:
        cost = data["cost"]
        reliability = data["reliability"]
        modified_weight = cost / reliability
        processed_edges.append((modified_weight, u, v, data))
    processed_edges.sort()

    network = CostReliabilityNetwork()
    network.add_nodes(nodes)
    components = {node: {node} for (node, _) in nodes}

    for _, u, v, data in processed_edges:
        comp_u = components[u]
        comp_v = components[v]

        can_add_edge = False
        if comp_u != comp_v:
            can_add_edge = True
        elif (
                network.get_node_reliability(u) < 1
                or network.get_node_reliability(v) < 1
        ):
            can_add_edge = True

        if can_add_edge:
            edge = (u, v, data)
            network.add_edge(edge)
            if comp_u != comp_v:
                new_component = comp_u.union(comp_v)
                for node in new_component:
                    components[node] = new_component

            if network.is_feasible:
                break

    return network


#### Metaheurística

In [78]:
def metaheuristic(
        topology: CostReliabilityGraph,
        max_iterations=1000,
        initial_temperature=100,
        cooling_rate=0.95,
) -> CostReliabilityNetwork:
    """
    Metaheuristic algorithm using Simulated Annealing.
    """
    nodes: list = topology.nodes
    all_edges: list = topology.edges

    # Start with an empty network
    current_network = CostReliabilityNetwork()
    current_network.add_nodes(nodes)
    best_network = current_network
    current_cost = float('inf')
    best_cost = current_cost

    T = initial_temperature
    T_min = 1e-3
    iteration = 0

    while T > T_min and iteration < max_iterations:
        # Copy current network
        new_network = CostReliabilityNetwork()
        new_network.add_nodes(nodes)
        new_network.add_edges(current_network.edges)

        # Attempt to make a move
        feasible_move_made = False
        attempts = 0
        max_attempts = 10  # To prevent infinite loops

        while not feasible_move_made and attempts < max_attempts:
            attempts += 1
            action = random.choice(["add", "remove", "replace"])
            if action == "add":
                edges_not_in_network = [
                    edge for edge in all_edges if not new_network.has_edge(edge[0], edge[1])
                ]
                if edges_not_in_network:
                    edge_to_add = random.choice(edges_not_in_network)
                    new_network.add_edge(edge_to_add)
                    if new_network.is_feasible:
                        feasible_move_made = True
                    else:
                        new_network.remove_edge(edge_to_add[0], edge_to_add[1])
            elif action == "remove":
                if len(new_network.edges) > 0:
                    edge_to_remove = random.choice(new_network.edges)
                    u, v, data = edge_to_remove
                    new_network.remove_edge(u, v)
                    if new_network.is_feasible:
                        feasible_move_made = True
                    else:
                        new_network.add_edge((u, v, data))
            elif action == "replace":
                if len(new_network.edges) > 0:
                    edge_to_remove = random.choice(new_network.edges)
                    u_remove, v_remove, data_remove = edge_to_remove
                    new_network.remove_edge(u_remove, v_remove)
                    edges_not_in_network = [
                        edge for edge in all_edges if not new_network.has_edge(edge[0], edge[1])
                    ]
                    if edges_not_in_network:
                        edge_to_add = random.choice(edges_not_in_network)
                        new_network.add_edge(edge_to_add)
                        if new_network.is_feasible:
                            feasible_move_made = True
                        else:
                            new_network.remove_edge(edge_to_add[0], edge_to_add[1])
                            new_network.add_edge((u_remove, v_remove, data_remove))
                    else:
                        new_network.add_edge((u_remove, v_remove, data_remove))

        # If no feasible move was made, proceed to the next iteration
        if not feasible_move_made:
            T *= cooling_rate
            iteration += 1
            continue

        # Calculate new cost
        new_cost = new_network.total_cost
        delta_cost = new_cost - current_cost

        # Acceptance criteria
        if delta_cost <= 0 or random.uniform(0, 1) < math.exp(-delta_cost / T):
            current_network = new_network
            current_cost = new_cost
            if new_cost < best_cost:
                best_network = new_network
                best_cost = new_cost

        T *= cooling_rate
        iteration += 1

    return best_network

## Comparación

In [79]:
import pandas as pd
import plotly.graph_objects as go

from src.experiment.benchmark import run_benchmarks

#### Parámetros

In [80]:
kernels = [brute_force, heuristic, metaheuristic]

input_sizes = list(range(3, 10))
inputs = [CostReliabilityGraph.generate_random_solvable_graph(n) for n in input_sizes]

timeout = 1 
num_samples = 3

#### Ejecución

In [81]:
benchmark_results = run_benchmarks(
    kernels=kernels,
    inputs=inputs,
    timeout=timeout,
    num_samples=num_samples
)

#### Post-procesamiento

##### Resultados iniciales

In [82]:
results = pd.DataFrame(benchmark_results)

In [83]:
results['cost'] = [r.total_cost if pd.notna(r) else None for r in results['result']]
results['reliability'] = [r.total_reliability if pd.notna(r) else None for r in results['result']]
results.drop(columns=['result'], inplace=True)

In [84]:
results = results.groupby(['kernel', 'input']).agg(
    {
        'time': ['mean', 'std'],
        'peak_memory': ['mean', 'std'],
        'cost': ['mean', 'std'],
        'reliability': ['mean', 'std']
    }
)

results.columns = ['_'.join(col).strip() for col in results.columns]

results.reset_index(inplace=True)

##### Atributos de entradas

In [85]:
inputs = [
    {
        "input": i,
        "size": input_sizes[i],
        "trivial_cost": input.total_cost,
        "trivial_reliability": input.total_reliability
    }
    for i, input in enumerate(inputs)
]

In [86]:
inputs = pd.DataFrame(inputs)

##### Resultados completos

In [87]:
results = pd.merge(inputs, results, on="input")
results.drop(columns=["input"], inplace=True)
results.insert(0, "kernel", results.pop("kernel"))

replace_dict = {"brute_force": "Fuerza Bruta", "heuristic": "Heurística", "metaheuristic": "Metaheurística"}
results["kernel"] = results["kernel"].replace(replace_dict)

In [88]:
results

,kernel,size,trivial_cost,trivial_reliability,time_mean,time_std,peak_memory_mean,peak_memory_std,cost_mean,cost_std,reliability_mean,reliability_std
0,Fuerza Bruta,3,37,2.705568,0.003331,0.000374,12668.333333,1946.477930,27.0,0.0,2.000000,0.0
1,Heurística,3,37,2.705568,0.004280,0.001881,12402.333333,1225.531041,37.0,0.0,2.705568,0.0
2,Metaheurística,3,37,2.705568,0.014177,0.000817,12785.666667,835.647254,0.0,0.0,0.000000,0.0
3,Fuerza Bruta,4,60,3.893279,0.005157,0.000379,20185.666667,995.091118,42.0,0.0,3.165411,0.0
4,Heurística,4,60,3.893279,0.003386,0.000890,19609.666667,1157.322917,45.0,0.0,3.418114,0.0
5,Metaheurística,4,60,3.893279,0.013886,0.000730,20470.333333,1160.124706,0.0,0.0,0.000000,0.0
6,Fuerza Bruta,5,123,5.117726,0.019902,0.000430,19450.666667,1551.260240,51.0,0.0,3.438774,0.0
7,Heurística,5,123,5.117726,0.003802,0.001681,18958.333333,1667.448750,51.0,0.0,3.438774,0.0
8,Metaheurística,5,123,5.117726,0.015873,0.001796,20127.333333,1068.011860,0.0,0.0,0.000000,0.0
9,Fuerza Bruta,6,166,6.166381,0.538780,0.048773,16816.000000,6917.561420,60.0,0.0,3.573948,0.0


#### Visualización

In [89]:
fig_time = go.Figure()

algorithms = ["Heurística", "Metaheurística"]
for algo in algorithms:
    df_algo = results[results['kernel'] == algo]
    fig_time.add_trace(go.Scatter(
        x=df_algo['size'],
        y=df_algo['time_mean'],
        mode='lines+markers',
        name=f'{algo}',
        error_y=dict(type='data', array=df_algo['time_std'], visible=True)
    ))

fig_time.update_layout(
    title='Tiempo de Ejecución vs. Tamaño de Entrada',
    xaxis_title='Tamaño de Entrada (n)',
    yaxis_title='Tiempo de Ejecución (s)',
    template='plotly_white'
)

fig_time.show()

In [90]:
fig_time = go.Figure()

algorithms = ["Fuerza Bruta"]
for algo in algorithms:
    df_algo = results[results['kernel'] == algo]
    fig_time.add_trace(go.Scatter(
        x=df_algo['size'],
        y=df_algo['time_mean'],
        mode='lines+markers',
        name=f'{algo}',
        error_y=dict(type='data', array=df_algo['time_std'], visible=True)
    ))

fig_time.update_layout(
    title='Tiempo de Ejecución vs. Tamaño de Entrada (Fuerza Bruta)',
    xaxis_title='Tamaño de Entrada (n)',
    yaxis_title='Tiempo de Ejecución (s)',
    template='plotly_white'
)

fig_time.show()

In [91]:
fig_time = go.Figure()

algorithms = results['kernel'].unique()
for algo in algorithms:
    df_algo = results[results['kernel'] == algo]
    fig_time.add_trace(go.Scatter(
        x=df_algo['size'],
        y=df_algo['peak_memory_mean'],
        mode='lines+markers',
        name=f'{algo}',
        error_y=dict(type='data', array=df_algo['peak_memory_std'], visible=True)
    ))

fig_time.update_layout(
    title='Memoria Pico vs. Tamaño de Entrada',
    xaxis_title='Tamaño de Entrada (n)',
    yaxis_title='Memoria Pico (B)',
    template='plotly_white'
)

fig_time.show()

In [92]:
fig_time = go.Figure()

algorithms = results['kernel'].unique()
for algo in algorithms:
    df_algo = results[results['kernel'] == algo]
    fig_time.add_trace(go.Scatter(
        x=df_algo['size'],
        y=df_algo['cost_mean'],
        mode='lines+markers',
        name=f'{algo}',
        error_y=dict(type='data', array=df_algo['cost_std'], visible=True)
    ))

fig_time.update_layout(
    title='Costo vs. Tamaño de Entrada',
    xaxis_title='Tamaño de Entrada (n)',
    yaxis_title='Costo',
    template='plotly_white'
)

fig_time.show()

In [93]:
fig_time = go.Figure()

algorithms = results['kernel'].unique()
for algo in algorithms:
    df_algo = results[results['kernel'] == algo]
    fig_time.add_trace(go.Scatter(
        x=df_algo['size'],
        y=df_algo['reliability_mean'],
        mode='lines+markers',
        name=f'{algo}',
        error_y=dict(type='data', array=df_algo['reliability_std'], visible=True)
    ))
    
fig_time.update_layout(
    title='Fiabilidad vs. Tamaño de Entrada',
    xaxis_title='Tamaño de Entrada (n)',
    yaxis_title='Fiabilidad',
    template='plotly_white'
)